In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression as LR
# from sklearn.ensemble import RandomForestRegressor as RFR
# from sklearn.model_selection import train_test_split,cross_val_score

In [2]:
df = pd.read_excel('./数据一组方案.xlsx',sheet_name='渠道')

In [3]:
df.columns

Index(['渠道', '日期', '点击', '下载', '注册', '下载→注册', '花费', '人均成本'], dtype='object')

In [4]:
df_prepare = df.drop(['日期', '点击', '下载', '下载→注册','人均成本'],axis=1)
df.groupby(by='渠道').mean()
X = df_prepare[['渠道','花费','注册']]

In [5]:
## 根据人均注册成本，样本划分为三箱
df_ios = X.query('渠道=="iOS"')
df_baidu_douyin = X.query('渠道=="抖音" or 渠道=="百度移动" or 渠道=="百度pc"')
df_other = X.query('渠道!="抖音" and 渠道!="百度移动" and 渠道!="百度pc" and 渠道!="iOS"')

print(df_ios.head())
print(df_baidu_douyin.head())
print(df_other.head())

       渠道         花费   注册
1825  iOS  640.29420  420
1826  iOS  660.20212  422
1827  iOS  682.65200  425
1828  iOS  684.71610  422
1829  iOS  728.50699  449
      渠道        花费  注册
2555  抖音  365.9740  20
2556  抖音  433.8960  24
2557  抖音  472.4252  26
2558  抖音  528.1277  29
2559  抖音  544.9530  30
   渠道         花费  注册
0  小米   830.0952  66
1  小米   936.6698  74
2  小米  1019.0447  79
3  小米   799.7566  62
4  小米   926.7580  70


In [6]:
## 渠道转数值型数据
dummies_baidu_douyin = pd.get_dummies(df_baidu_douyin.渠道)
dummies_other = pd.get_dummies(df_other.渠道)
dummy_ios = pd.get_dummies(df_ios.渠道)

In [7]:
## 将2019-01-01——2019-12-31 写成 1——365 以便于建立模型
from numpy import array
time_ios = np.arange(1,366,1)
time_3 = array([time_ios,time_ios,time_ios]).flatten()
time_6= array([time_3,time_3]).flatten()
dummy_ios['date'] = time_ios
dummies_baidu_douyin['date']= time_3
dummies_other['date'] = time_6

In [8]:
X_baidu_douyin = pd.concat([df_baidu_douyin[['花费']],dummies_baidu_douyin],axis=1)
X_other = pd.concat([df_other[['花费']],dummies_other],axis=1)
X_ios = pd.concat([df_ios[["花费"]],dummy_ios],axis=1)

In [9]:
print(X_ios.head())
print(X_baidu_douyin.head())
print(X_other.head())

             花费  iOS  date
1825  640.29420    1     1
1826  660.20212    1     2
1827  682.65200    1     3
1828  684.71610    1     4
1829  728.50699    1     5
            花费  抖音  百度pc  百度移动  date
2555  365.9740   1     0     0     1
2556  433.8960   1     0     0     2
2557  472.4252   1     0     0     3
2558  528.1277   1     0     0     4
2559  544.9530   1     0     0     5
          花费  oppo  vivo  华为  小米  快手  魅族  date
0   830.0952     0     0   0   1   0   0     1
1   936.6698     0     0   0   1   0   0     2
2  1019.0447     0     0   0   1   0   0     3
3   799.7566     0     0   0   1   0   0     4
4   926.7580     0     0   0   1   0   0     5


In [10]:
y_ios = df_ios.注册
y_baidu_douyin = df_baidu_douyin.注册
y_other = df_other.注册

In [11]:
## ios
y_ios_time = y_ios 
## douyin_baidu
y_douyin_time = df_baidu_douyin.query('渠道=="抖音"').注册
y_baidupc_time = df_baidu_douyin.query('渠道=="百度pc"').注册
y_baidumobile_time = df_baidu_douyin.query('渠道=="百度移动"').注册
## other
y_oppo_time = df_other.query('渠道=="oppo"').注册
y_vivo_time = df_other.query('渠道=="vivo"').注册
y_hw_time = df_other.query('渠道=="华为"').注册
y_mi_time = df_other.query('渠道=="小米"').注册
y_kuai_time = df_other.query('渠道=="快手"').注册
y_meizu_time = df_other.query('渠道=="魅族"').注册

In [12]:
## ios
lr_ios_time = LR()
lr_ios_time.fit(time_ios.reshape(-1,1),y_ios_time)
## 抖音
lr_douyin_time = LR()
lr_douyin_time.fit(time_ios.reshape(-1,1),y_douyin_time)
## 百度PC
lr_baidupc_time = LR()
lr_baidupc_time.fit(time_ios.reshape(-1,1),y_baidupc_time)
## 百度移动
lr_baidumobile_time = LR()
lr_baidumobile_time.fit(time_ios.reshape(-1,1),y_baidumobile_time)
## oppo
lr_oppo_time = LR()
lr_oppo_time.fit(time_ios.reshape(-1,1),y_oppo_time)
## vivo
lr_vivo_time = LR()
lr_vivo_time.fit(time_ios.reshape(-1,1),y_vivo_time)
## 华为
lr_hw_time = LR()
lr_hw_time.fit(time_ios.reshape(-1,1),y_hw_time)
## 小米
lr_mi_time = LR()
lr_mi_time.fit(time_ios.reshape(-1,1),y_mi_time)
## 快手
lr_kuai_time = LR()
lr_kuai_time.fit(time_ios.reshape(-1,1),y_kuai_time)
## 魅族
lr_meizu_time = LR()
lr_meizu_time.fit(time_ios.reshape(-1,1),y_meizu_time)

print('ios:',lr_ios_time.score(time_ios.reshape(-1,1),y_ios))
print('douyin:',lr_douyin_time.score(time_ios.reshape(-1,1),y_douyin_time))
print('baidupc:',lr_baidupc_time.score(time_ios.reshape(-1,1),y_baidupc_time))
print('baidumobile:',lr_baidumobile_time.score(time_ios.reshape(-1,1),y_baidumobile_time))
print('oppo:',lr_oppo_time.score(time_ios.reshape(-1,1),y_oppo_time))
print('vivo:',lr_vivo_time.score(time_ios.reshape(-1,1),y_vivo_time))
print('huawei:',lr_hw_time.score(time_ios.reshape(-1,1),y_hw_time))
print('mi:',lr_mi_time.score(time_ios.reshape(-1,1),y_mi_time))
print('kuai:',lr_kuai_time.score(time_ios.reshape(-1,1),y_kuai_time))
print('meizu:',lr_meizu_time.score(time_ios.reshape(-1,1),y_meizu_time))

ios: 0.9072365466991168
douyin: 0.8436889418708342
baidupc: 0.6099256283575808
baidumobile: 0.7290388504668446
oppo: 0.9619167233904851
vivo: 0.8998802844401323
huawei: 0.886526732281549
mi: 0.8627659060454727
kuai: 0.9115852819474795
meizu: 0.6219259536058868


In [13]:
## 预测双11未增投时注册量
print('预测双11未增投时注册量:')
print('ios:',lr_ios_time.predict([[675]]))
print('抖音:',lr_douyin_time.predict([[675]]))
print('百度pc:',lr_baidupc_time.predict([[675]]))
print('百度移动:',lr_baidumobile_time.predict([[675]]))
print('oppo:',lr_oppo_time.predict([[675]]))
print('vivo:',lr_vivo_time.predict([[675]]))
print('华为:',lr_hw_time.predict([[675]]))
print('小米:',lr_mi_time.predict([[675]]))
print('快手:',lr_kuai_time.predict([[675]]))
print('魅族:',lr_meizu_time.predict([[675]]))

预测双11未增投时注册量:
ios: [2487.11022968]
抖音: [252.00173189]
百度pc: [38.69853192]
百度移动: [531.48864058]
oppo: [1149.89732073]
vivo: [1132.94060801]
华为: [1545.56919375]
小米: [557.17209685]
快手: [357.45935004]
魅族: [76.23713955]


In [14]:
## 计算10%增长和15增长后的注册数
ios_10 = lr_ios_time.predict([[675]])*1.1
douyin_10 = lr_douyin_time.predict([[675]])*1.1
baidupc_10 = lr_baidupc_time.predict([[675]])*1.1
baidumobile_10 = lr_baidumobile_time.predict([[675]])*1.1
oppo_10 = lr_oppo_time.predict([[675]])*1.1
vivo_10 = lr_vivo_time.predict([[675]])*1.1
hw_10 = lr_hw_time.predict([[675]])*1.1
mi_10 = lr_mi_time.predict([[675]])*1.1
kuai_10 = lr_kuai_time.predict([[675]])*1.1
meizu_10 = lr_meizu_time.predict([[675]])*1.1

ios_15 = lr_ios_time.predict([[675]])*1.15
douyin_15 = lr_douyin_time.predict([[675]])*1.15
baidupc_15 = lr_baidupc_time.predict([[675]])*1.15
baidumobile_15 = lr_baidumobile_time.predict([[675]])*1.15
oppo_15 = lr_oppo_time.predict([[675]])*1.15
vivo_15 = lr_vivo_time.predict([[675]])*1.15
hw_15 = lr_hw_time.predict([[675]])*1.15
mi_15 = lr_mi_time.predict([[675]])*1.15
kuai_15 = lr_kuai_time.predict([[675]])*1.15
meizu_15 = lr_meizu_time.predict([[675]])*1.15


In [15]:
## 时间，花费，渠道 —— 预测注册
lr_ios_regist = LR()
lr_ios_regist.fit(X_ios,y_ios)
lr_douyinbaidu_regist = LR()
lr_douyinbaidu_regist.fit(X_baidu_douyin,y_baidu_douyin)
lr_other_regist = LR()
lr_other_regist.fit(X_other,y_other)

print(lr_ios_regist.score(X_ios,y_ios))
print(lr_douyinbaidu_regist.score(X_baidu_douyin,y_baidu_douyin))
print(lr_other_regist.score(X_other,y_other))

0.9650885918336864
0.9981416087403896
0.9941149627121019


In [16]:
## 增投10%
## ios
for i in range(4500,7000,100):
    d = lr_ios_regist.predict([[i,1,675]])-ios_10
    if d[0]>0:
        print('ios:',i)
        break
## oppo
for i in range(11000,17000,500):
    d = lr_other_regist.predict([[i,1,0,0,0,0,0,675]])-oppo_10
    if d[0]>0:
        print('oppo:',i)
        break
## vivo
for i in range(11000,19000,500):
    d = lr_other_regist.predict([[i,0,1,0,0,0,0,675]])-vivo_10
    if d[0]>0:
        print('vivo:',i)
        break
## 华为
for i in range(15000,25000,500):
    d = lr_other_regist.predict([[i,0,0,1,0,0,0,675]])-hw_10
    if d[0]>0:
        print('hw:',i)
        break
## 小米
for i in range(6000,12000,200):
    d = lr_other_regist.predict([[i,0,0,0,1,0,0,675]])-mi_10
    if d[0]>0:
        print('mi:',i)
        break
## 快手
for i in range(3500,8000,100):
    d = lr_other_regist.predict([[i,0,0,0,0,1,0,675]])-kuai_10
    if d[0]>0:
        print('kuai:',i)
        break
## 抖音
for i in range(4500,8000,100):
    d = lr_douyinbaidu_regist.predict([[i,1,0,0,675]])-douyin_10
    if d[0]>0:
        print('douyin:',i)
        break
## 百度pc
for i in range(800,3000,100):
    d = lr_douyinbaidu_regist.predict([[i,0,0,1,675]])-baidupc_10
    if d[0]>0:
        print('baidupc:',i)
        break
## 百度移动
for i in range(10000,15000,100):
    d = lr_douyinbaidu_regist.predict([[i,0,1,0,675]])-baidumobile_10
    if d[0]>0:
        print('baidumobile:',i)
        break
## 魅族
for i in range(800,1100,100):
    d = lr_other_regist.predict([[i,0,0,0,0,0,1,675]])-meizu_10
    if d[0]>0:
        print('meizu:',i)
        break

ios: 5900
oppo: 15000
vivo: 15500
hw: 21000
mi: 7400
kuai: 4400
douyin: 5600
baidupc: 800
baidumobile: 11900
meizu: 800


In [17]:
## 增投15%
## ios
for i in range(4500,7000,100):
    d = lr_ios_regist.predict([[i,1,675]])-ios_15
    if d[0]>0:
        print('ios:',i)
        break
## oppo
for i in range(11000,17000,500):
    d = lr_other_regist.predict([[i,1,0,0,0,0,0,675]])-oppo_15
    if d[0]>0:
        print('oppo:',i)
        break
## vivo
for i in range(11000,19000,500):
    d = lr_other_regist.predict([[i,0,1,0,0,0,0,675]])-vivo_15
    if d[0]>0:
        print('vivo:',i)
        break
## 华为
for i in range(15000,25000,500):
    d = lr_other_regist.predict([[i,0,0,1,0,0,0,675]])-hw_15
    if d[0]>0:
        print('hw:',i)
        break
## 小米
for i in range(6000,12000,200):
    d = lr_other_regist.predict([[i,0,0,0,1,0,0,675]])-mi_15
    if d[0]>0:
        print('mi:',i)
        break
## 快手
for i in range(3500,8000,100):
    d = lr_other_regist.predict([[i,0,0,0,0,1,0,675]])-kuai_15
    if d[0]>0:
        print('kuai:',i)
        break
## 抖音
for i in range(4500,8000,100):
    d = lr_douyinbaidu_regist.predict([[i,1,0,0,675]])-douyin_15
    if d[0]>0:
        print('douyin:',i)
        break
## 百度pc
for i in range(800,3000,100):
    d = lr_douyinbaidu_regist.predict([[i,0,0,1,675]])-baidupc_15
    if d[0]>0:
        print('baidupc:',i)
        break
## 百度移动
for i in range(10000,15000,100):
    d = lr_douyinbaidu_regist.predict([[i,0,1,0,675]])-baidumobile_15
    if d[0]>0:
        print('baidumobile:',i)
        break
## 魅族
for i in range(800,1100,100):
    d = lr_other_regist.predict([[i,0,0,0,0,0,1,675]])-meizu_15
    if d[0]>0:
        print('meizu:',i)
        break

ios: 6300
oppo: 16000
vivo: 16000
hw: 22000
mi: 7800
kuai: 4600
douyin: 5800
baidupc: 800
baidumobile: 12400
meizu: 800


In [18]:
#### 15%达不到，尝试12.5%

In [19]:
ios_125 = lr_ios_time.predict([[675]])*1.125
douyin_125 = lr_douyin_time.predict([[675]])*1.125
baidupc_125 = lr_baidupc_time.predict([[675]])*1.125
baidumobile_125 = lr_baidumobile_time.predict([[675]])*1.125
oppo_125 = lr_oppo_time.predict([[675]])*1.125
vivo_125 = lr_vivo_time.predict([[675]])*1.125
hw_125 = lr_hw_time.predict([[675]])*1.125
mi_125 = lr_mi_time.predict([[675]])*1.125
kuai_125 = lr_kuai_time.predict([[675]])*1.125
meizu_125 = lr_meizu_time.predict([[675]])*1.125

In [20]:
## 增投12.5% 后
## ios
for i in range(4500,7000,100):
    d = lr_ios_regist.predict([[i,1,675]])-ios_125
    if d[0]>0:
        print('ios:',i,'元')
        break
## oppo
for i in range(11000,17000,500):
    d = lr_other_regist.predict([[i,1,0,0,0,0,0,675]])-oppo_125
    if d[0]>0:
        print('oppo:',i,'元')
        break
## vivo
for i in range(11000,19000,500):
    d = lr_other_regist.predict([[i,0,1,0,0,0,0,675]])-vivo_125
    if d[0]>0:
        print('vivo:',i,'元')
        break
## 华为
for i in range(15000,25000,500):
    d = lr_other_regist.predict([[i,0,0,1,0,0,0,675]])-hw_125
    if d[0]>0:
        print('hw:',i,'元')
        break
## 小米
for i in range(6000,12000,200):
    d = lr_other_regist.predict([[i,0,0,0,1,0,0,675]])-mi_125
    if d[0]>0:
        print('mi:',i,'元')
        break
## 快手
for i in range(3500,8000,100):
    d = lr_other_regist.predict([[i,0,0,0,0,1,0,675]])-kuai_125
    if d[0]>0:
        print('kuai:',i,'元')
        break
## 抖音
for i in range(4500,8000,100):
    d = lr_douyinbaidu_regist.predict([[i,1,0,0,675]])-douyin_125
    if d[0]>0:
        print('douyin:',i,'元')
        break
## 百度pc
for i in range(800,3000,100):
    d = lr_douyinbaidu_regist.predict([[i,0,0,1,675]])-baidupc_125
    if d[0]>0:
        print('baidupc:',i,'元')
        break
## 百度移动
for i in range(10000,15000,100):
    d = lr_douyinbaidu_regist.predict([[i,0,1,0,675]])-baidumobile_125
    if d[0]>0:
        print('baidumobile:',i,'元')
        break
## 魅族
for i in range(800,1100,100):
    d = lr_other_regist.predict([[i,0,0,0,0,0,1,675]])-meizu_125
    if d[0]>0:
        print('meizu:',i,'元')
        break

ios: 6100 元
oppo: 15500 元
vivo: 15500 元
hw: 21500 元
mi: 7600 元
kuai: 4500 元
douyin: 5700 元
baidupc: 800 元
baidumobile: 12200 元
meizu: 800 元
